In [1]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
app = Flask(__name__)

# Load the trained model
model = joblib.load('trained_model.pkl')  # Replace with your model file path


In [4]:
# Define preprocessing
def preprocess_input(data):
     \print("Raw Data", data)
    competitor_columns = data.filter(like='competitor').columns
    data['max_discount'] = np.round(data['actual_price'] - data[competitor_columns].min(axis=1), 2)

    data['mmin_price'] = data[competitor_columns].min(axis=1)
    numeric_features = [col for col in data.columns if pd.api.types.is_numeric_dtype(data[col])]
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', 'passthrough', ['product_type'])  # Assuming 'product_type' is categorical
        ])
    print("Transformed Data", data)
    return preprocessor.transform(data)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [ ]:
# Endpoint to predict promotional price
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get JSON request data
        json_data = request.get_json()

        # Convert JSON to DataFrame
        input_data = pd.DataFrame([json_data])

        # Preprocess input data
        X_input = preprocess_input(input_data)

        # Predict with the model
        predicted_price = model.predict(X_input)[0]

        # Prepare response
        response = {
            'predicted_promotional_price': predicted_price
        }

        return jsonify(response)

    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True, port=6767, use_reloader=False)

In [ ]:
%tb